In [3]:
from main import Eval_Object_Detector

path = '/data2/iena/241030_SKMagic_YOLO/1_preprocess'
path_main = f'{path}/8_기타 테스트/250411_PR계산 구현/Object_Detector_Evaluation'


path_gt = f'{path}/8_기타 테스트/250411_PR계산 구현/test_data/gt'
path_pred = f'{path}/8_기타 테스트/250411_PR계산 구현/test_data/pred'
class_list = ['cable', 'carpet', 'clothes', 'obstacle', 'poop']
path_output = f'{path_main}/output'

path_img = f'{path}/8_기타 테스트/250411_PR계산 구현/test_data/images'

In [2]:
mAP = Eval_Object_Detector(path_gt, path_pred, class_list, path_output)
mAP.evaluate()

코드 릴리즈 날짜: 250416_v1
evaluate() 이용 시 전체적인 평가가 가능합니다.
1회 평가 이후 draw_bgfp_auto(path_img) 이용 시 Background FP에 대한 사례들을 그려볼 수 있습니다.



Precision, Recall 계산 중...: 100%|██████████| 1000/1000 [00:14<00:00, 70.64it/s]


Confusion Matrix saved to: /data2/iena/241030_SKMagic_YOLO/1_preprocess/8_기타 테스트/250411_PR계산 구현/Object_Detector_Evaluation/output/Confusion_Matrix.png


mAP@0.5:0.95 계산 중...: 100%|██████████| 10/10 [02:33<00:00, 15.40s/it]


=== Evaluation Results ===

   Class  Images  Labels        P        R  mAP@.5  mAP@.5:.95  F1_score  Conf. Thr.
     all    5224    4073    0.685    0.671   0.669       0.506     0.668       0.352
   cable     638     688    0.935    0.671   0.813       0.562     0.781       0.513
  carpet     724     724    0.818    0.802   0.815       0.744     0.810       0.314
 clothes     278     278    0.530    0.791   0.687       0.603     0.635       0.440
obstacle    2039    2261    0.706    0.633   0.657       0.415     0.668       0.251
    poop     122     122    0.438    0.459   0.375       0.204     0.448       0.241

CSV saved to: /data2/iena/241030_SKMagic_YOLO/1_preprocess/8_기타 테스트/250411_PR계산 구현/Object_Detector_Evaluation/output/evaluation_summary.csv


In [4]:
mAP.draw_bgfp_auto(path_img)

100%|██████████| 9/9 [02:07<00:00, 14.15s/it]
